In [1]:
import pandas as pd
import pickle

In [2]:
tweet=pd.read_pickle('NhungPham_EN.pkl')
sentiment=pd.read_pickle('Trang-Nhung Pham - NhungPham_EN_completed.pkl')
retweet=pd.read_pickle('retweets.pkl')

In [3]:
def word_present(text, query):
    import re
    text = str(text).lower()
    newquery = []
    for word in query:
        newquery.append(str(word).lower())
    tokens = re.findall(r"[\w']+|[.,!?;$@#]", text)
    
    for word in newquery:
        if word in tokens:
            return 1
    return 0

In [4]:
tweet['data']=tweet['text'].apply(word_present,args=(['data']))
tweet['social media']=tweet['text'].apply(word_present,args=(['social media']))
tweet['online']=tweet['text'].apply(word_present,args=(['online']))

In [5]:
def categorise_df(row):
    row['text includes'] = "text includes"
    if (row['data'] == 1) and (row['social media'] == 1) and (row['online'] == 1):
        row['text includes'] = 'All'
    if (row['data'] == 1) and (row['social media'] == 0) and (row['online'] == 0):
        row['text includes'] = 'data'
    if (row['data'] == 0) and (row['social media'] == 1) and (row['online'] == 0):
        row['text includes'] = 'social media'
    if (row['data'] == 0) and (row['social media'] == 0) and (row['online'] == 1):
        row['text includes'] = 'online'
    if (row['data'] == 1) and (row['social media'] == 1) and (row['online'] == 0):
        row['text includes'] = 'data and social media'
    if (row['data'] == 1) and (row['social media'] == 0) and (row['online'] == 1):
        row['text includes'] = 'data and online'
    if (row['data'] == 0) and (row['social media'] == 1) and (row['online'] == 1):
        row['text includes'] = 'social media and online'
    if (row['data'] == 0) and (row['social media'] == 0) and (row['online'] == 0):
        row['text includes'] = 'Nothing'
    return row

In [6]:
tweet=tweet.apply(categorise_df,axis=1)

In [7]:
tweet['text includes'].value_counts()

data                       311
data and social media       42
All                         13
Nothing                      6
social media                 1
social media and online      1
Name: text includes, dtype: int64

In [8]:
tweet.groupby('text includes', as_index=False)['from_user_followercount'].mean()

,text includes,from_user_followercount
0,All,1555.461538
1,Nothing,342.333333
2,data,2433.453376
3,data and social media,7842.619048
4,social media,7159.000000
5,social media and online,3247.000000


Above: comparing means of followers by categories

In [9]:
sentiment['negative']=sentiment['negative'].apply(pd.to_numeric)
sentiment['positive']=sentiment['positive'].apply(pd.to_numeric)
sentiment['neutral']=sentiment['neutral'].apply(pd.to_numeric)
sentiment['sentiment']=sentiment['negative']+sentiment['positive']+sentiment['neutral']

In [10]:
data=pd.merge(tweet,sentiment,how='inner',on=['id','text'])

In [11]:
data.shape

(13, 44)

In [12]:
data.groupby('text includes', as_index=False)['sentiment'].mean()

,text includes,sentiment
0,Nothing,2.000000
1,data,0.090909
2,data and social media,0.000000


Above: comparing sentiment by categories. As the problem of reduced data hasn't been solved, data here is much less than it should be

In [13]:
retweet=retweet.rename(columns = {'retweet_id':'id'})

In [14]:
def is_rt(text):
    text = text.split()
    if text[0] == 'RT':
        return 1
    return 0

In [15]:
tweet['is_retweet'] = tweet['text'].apply(is_rt)

In [16]:
only_tweet=tweet[tweet['is_retweet']==0]

In [17]:
full_tweet=pd.merge(only_tweet,retweet,how='left', on='id')
full_tweet['retweets_total']=full_tweet['retweets_total'].fillna(0)

In [18]:
full_tweet

,id,time,created_at,from_user_name,text,filter_level,possibly_sensitive,withheld_copyright,withheld_scope,truncated,...,from_user_favourites_count,from_user_listed,from_user_withheld_scope,from_user_created_at,data,social media,online,text includes,is_retweet,retweets_total
0,828544758989602048,1486375438,2017-02-06 10:03:58,MarketingJobs4U,"Senior Digital Analytics Manager - £70,000 (+1...",low,0.0,NaN,NaN,0,...,0,249,NaN,2013-02-19 17:41:37,1,0,0,data,0,0.0
1,828553689140980992,1486377567,2017-02-06 10:39:27,guzogang,"Senior Digital Analytics Manager - £70,000 (+1...",low,0.0,NaN,NaN,0,...,2,336,NaN,2009-05-30 22:48:12,1,0,0,data,0,0.0
2,828556421822311936,1486378219,2017-02-06 10:50:19,ShikhaBakshi,Fractal Analytics is hiring a Principal Consul...,low,0.0,NaN,NaN,0,...,2,24,NaN,2012-09-25 06:57:33,1,1,0,data and social media,0,0.0
3,828559249773059968,1486378893,2017-02-06 11:01:33,DIMHouston,Test Your Mettle: Adobe Digital Analytics Comp...,low,0.0,NaN,NaN,0,...,869,152,NaN,2014-07-21 18:11:59,1,0,0,data,0,0.0
4,828576889560109056,1486383099,2017-02-06 12:11:39,Optizent,Digital Analytics Speakers Required https://t....,low,0.0,NaN,NaN,0,...,6,97,NaN,2009-03-13 05:15:28,1,0,0,data,0,0.0
5,828619425419030016,1486393240,2017-02-06 15:00:40,HeadHunterJames,Senior Analyst (Digital Analytics) - Rogers Co...,low,0.0,NaN,NaN,0,...,9,9,NaN,2010-01-08 16:48:19,1,0,0,data,0,0.0
6,828623455239036032,1486394201,2017-02-06 15:16:41,tmj_aus_adv,Can you recommend anyone for this #job? Senior...,low,0.0,NaN,NaN,0,...,0,335,NaN,2009-03-12 03:53:42,1,0,0,data,0,0.0
7,828655123018632960,1486401751,2017-02-06 17:22:31,Optizent,Senior Digital Analytics Specialist at Consume...,low,0.0,NaN,NaN,0,...,6,97,NaN,2009-03-13 05:15:28,1,0,0,data,0,0.0
8,828660295165997056,1486402984,2017-02-06 17:43:04,nicolemleite,Digital Analytics Weekly Update is out! https:...,low,0.0,NaN,NaN,0,...,87,99,NaN,2011-10-31 03:21:38,1,0,0,data,0,0.0
9,828664538157424000,1486403996,2017-02-06 17:59:56,TopTalentMatch,Wanted: Digital Analytics talents to bring in ...,low,0.0,NaN,NaN,0,...,32,1,NaN,2009-12-19 14:21:27,1,1,0,data and social media,0,0.0


Here the problem is clear: Id differs among the table (even though they should be definitely the same). However hopefully the code above works in merging tweets and retweets and therefore works well for the requirement.

In [19]:
tweet=pd.read_pickle('NhungPham_EN.pkl')
sentiment=pd.read_pickle('Trang-Nhung Pham - NhungPham_EN_completed.pkl')


In [23]:
len(tweet.merge(sentiment))

13

In [25]:
tweet['id'].describe()

count    3.740000e+02
mean     8.299913e+17
std      9.531119e+14
min      8.285404e+17
25%      8.291933e+17
50%      8.298195e+17
75%      8.308925e+17
max      8.321202e+17
Name: id, dtype: float64

In [27]:
sentiment['id'].describe()

count    3.740000e+02
mean     8.299913e+17
std      9.531119e+14
min      8.285404e+17
25%      8.291933e+17
50%      8.298195e+17
75%      8.308925e+17
max      8.321202e+17
Name: id, dtype: float64

In [28]:
tweet.iloc[0]['id']

828540415360003968

In [30]:
sentiment.iloc[1]['id']

828540731975401473

In [32]:
ids_tweet = tweet['id'].values.tolist()

In [33]:
ids_sent = sentiment['id'].values.tolist()

In [35]:
set(ids_sent) - set(ids_tweet)

{828540415360004096,
 828540731975401473,
 828540960443351041,
 828544758989602816,
 828553689140981760,
 828556421822312449,
 828559249773060096,
 828565053674749952,
 828565322907131904,
 828565538926362624,
 828576889560109057,
 828619425419030529,
 828623455239036929,
 828655123018633216,
 828664538157424641,
 828670925893664769,
 828671243708665857,
 828678402936803329,
 828699952360468480,
 828704074983960578,
 828720830918033408,
 828732615284891648,
 828734517179359232,
 828739103135059972,
 828742906643103745,
 828749693404405760,
 828777463400583172,
 828793206263336962,
 828813422452092934,
 828843349582897152,
 828845533938716672,
 828858718326435840,
 828861837580046336,
 828871692432838657,
 828871703333892096,
 828871827606745089,
 828877677092298752,
 828879934936514560,
 828879938820386817,
 828884864791371776,
 828891073460187136,
 828944064905084929,
 828944065001578496,
 828944065387438081,
 828944065404211200,
 828944065454600195,
 828951956198670336,
 828961065773